In [2]:
import math

import numpy as np
from cartopy import crs
from cartopy.feature import COLORS, NaturalEarthFeature
from matplotlib import pyplot
from matplotlib.cm import get_cmap
from matplotlib.colors import from_levels_and_colors
from netCDF4 import Dataset
from scipy.interpolate import griddata
from wrf import (
    CoordPair,
    cartopy_xlim,
    cartopy_ylim,
    get_cartopy,
    getvar,
    interpline,
    latlon_coords,
    to_np,
    vertcross,
)

In [3]:
##function def part
#cross_start = CoordPair(lat=41.2, lon=-106.56)
#cross_end = CoordPair(lat=41.4, lon=-105.97)
###getvar from wrf files
def get(wrf_file,times):
    u = getvar(wrf_file, "ua",  timeidx = times)
    v = getvar(wrf_file, "va", timeidx = times)
    w = getvar(wrf_file, "wa", timeidx = times)
    ht = getvar(wrf_file, "z", timeidx=times)
    ter = getvar(wrf_file, "ter", timeidx=times)
    dbz = getvar(wrf_file, "REFL_10CM", timeidx=times)
    dbz = 10**(dbz/10)
    vice = getvar(wrf_file, "V_ICE", timeidx=times)
    nice = getvar(wrf_file, "NISLF", timeidx=times)*3600
    qc_d = getvar(wrf_file, "QCHETC", timeidx=times)*3600000
    qi_d = getvar(wrf_file, "QCHETI", timeidx=times)*3600000
    qc_r = getvar(wrf_file, "QRHETC", timeidx=times)*3600000
    qi_r = getvar(wrf_file, "QRHETI", timeidx=times)*3600000
    return u, v, w, ht, ter, dbz, vice, nice, qc_d, qi_d, qc_r, qi_r
## let the values under filled shape disappeared.
def cross(x,des,uni):
  #  X = 10**(x/10)
    x_cross = vertcross(x, ht, wrfin=wrf_file_c,start_point=cross_start,end_point=cross_end,latlon=True,meta=True)
    x1_cross = x_cross#10.0 * np.log10(x_cross)
    x1_cross.attrs.update(x_cross.attrs)
    x1_cross.attrs["description"] = des
    x1_cross.attrs["units"] = uni
    x_cross_filled = np.ma.copy(to_np(x1_cross))
    for i in range(x_cross_filled.shape[-1]):
        column_vals = x_cross_filled[:,i]
        # Let's find the lowest index that isn't filled. The nonzero function
        # finds all unmasked values greater than 0. Since 0 is a valid value
        # for dBZ, let's change that threshold to be -200 dBZ instead.
        first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
        x_cross_filled[0:first_idx, i] = 0
    return x_cross_filled,x1_cross
##Calculate the relative diff
def diff(c, t):
    acc_percent = np.empty(shape = np.shape(c))
    acc_percent[:, :] = t[:, :]-c[:, :]
    acc_percent[np.isinf(acc_percent)]=0
    acc_percent = np.nan_to_num(acc_percent)
    return acc_percent

In [4]:
## i is the index of date, change to 18, 19 means change to date 18.
for i in range(19, 20):
    for j in range(0,1):
        if i==18:
            times_t = j
            times_c = j
            timeout = j+12
        elif i==19:
            times_t = j
            times_c = j
            timeout = j
       #wrf_file_t = Dataset("/glade/scratch/mingzhu/High_res/WRF/300out_turb_sierra/wrfout_d03_2006-01-18_12:00:00")
       #wrf_file_c = Dataset("/glade/scratch/mingzhu/High_res/WRF/300out_cont_sierra/wrfout_d03_2006-01-18_12:00:00")
        wrf_file_t = Dataset("/glade/scratch/mingzhu/High_res/WRF_test/run_ysu_turb/wrfout_d03_2006-01-19_00:00:00")
        wrf_file_c = Dataset("/glade/scratch/mingzhu/High_res/WRF_test/run_ysu_cont/wrfout_d03_2006-01-19_00:00:00")
        angle = math.radians(14)
        u, v, w, ht, ter, dbz, vice, nice, qc_d, qi_d, qc_r, qi_r = get(wrf_file_c,times_c)
        u_t, v_t, w_t, ht_t, ter_t, dbz_t, vice_t, nice_t, qc_d_t, qi_d_t, qc_r_t, qi_r_t = get(wrf_file_t,times_t)
        ####for define the CONT value's unit and description#####
        dbz,dbz1 = cross(dbz,'reflectivity', 'dBZ')
        ua = cross(u,'U-wind', 'm/s')[0]
        va = cross(v,'V-wind', 'm/s')[0]
        wa = cross(w,'W-wind', 'm/s')[0]
        vice = cross(vice, 'Vice', 'm/s')[0]
        nice = cross(nice,'change in ice number from collection','kg-1 hr-1')[0]
        qc_d = cross(qc_d,'contact freezing droplet','g kg-1 hr-1')[0]
        qi_d = cross(qi_d,'immersion freezing droplet','g kg-1 hr-1')[0]
        qc_r = cross(qc_r,'contact freezing rain','g kg-1 hr-1')[0]
        qi_r = cross(qi_r, 'immersion freezing rain', 'g kg-1 hr-1')[0]
        ####for define the TURB value's unit and description#####
        dbz_t,dbz1_t = cross(dbz_t,'reflectivity', 'dBZ')
        ua_t = cross(u_t,'U-wind', 'm/s')[0]
        va_t = cross(v_t,'V-wind', 'm/s')[0]
        wa_t = cross(w_t,'W-wind', 'm/s')[0]
        vice_t = cross(vice_t, 'Vice', 'm/s')[0]
        nice_t = cross(nice_t,'change in ice number from collection','kg-1 hr-1')[0]
        qc_d_t = cross(qc_d_t,'contact freezing droplet','g kg-1 hr-1')[0]
        qi_d_t = cross(qi_d_t,'immersion freezing droplet','g kg-1 hr-1')[0]
        qc_r_t = cross(qc_r_t,'contact freezing rain','g kg-1 hr-1')[0]
        qi_r_t = cross(qi_r_t, 'immersion freezing rain', 'g kg-1 hr-1')[0]
        ####for calculate the doppler velocity and stream line##
        #the cross_section is not parralell to the x-axis, so the stream line should
        #multiplied by angle between cross_section and x-axis
        dopp = to_np(wa-vice)
        hori = ua * math.cos(angle)
        stream = to_np(ua * math.cos(angle) + va * math.sin(angle))
        dopp_t = to_np(wa_t-vice_t)
        hori_t = ua_t * math.cos(angle)
        stream_t = to_np(ua_t * math.cos(angle) + va_t * math.sin(angle))
        #####For calculate the relative difference##############
        dbz_diff = diff(dbz, dbz_t)
        dopp_diff = diff(dopp, dopp_t)
        hori_diff = diff(hori,hori_t)
        nice_diff = diff(nice, nice_t)
        qc_d_diff = diff(qc_d, qc_d_t)
        qi_d_diff = diff(qi_d, qi_d_t)
        qc_r_diff = diff(qc_r, qc_r_t)
        qi_r_diff = diff(qi_r, qi_r_t)
        #####draw the terrain shape#############################
        ter_line = interpline(ter, wrfin=wrf_file_c, start_point=cross_start,
                      end_point=cross_end)
        lats, lons = latlon_coords(u)#x, y coord
        cart_proj = get_cartopy(u)#import the project from wrf value
        ###creat the figure##################################3
        fig, ((ax_nice,ax_nice_t,ax_nice_diff)#qcloud
             ,(ax_qc_d, ax_qc_d_t, ax_qc_d_diff)#qccol
             ,(ax_qi_d, ax_qi_d_t, ax_qi_d_diff)#QICE
             ,(ax_qc_r, ax_qc_r_t, ax_qc_r_diff)#DEp
             ,(ax_qi_r, ax_qi_r_t, ax_qi_r_diff)#QRIM
             ) = pyplot.subplots(5,3,figsize=(12,10),constrained_layout=True)
        xs = np.arange(0, dbz1.shape[-1], 1)*300
        ys = to_np(dbz1.coords["vertical"])
    
        def crossplot(cname, x, y, z, hori, ver, levels, colors, title):
                      var_fill = cname.contourf(x,y,z, levels = levels, cmap=get_cmap(colors), extend = 'both')
                      ht_fill = cname.fill_between(x, 0, to_np(ter_line),facecolor="saddlebrown")
                      cname.set_ylim([0, 7000])
                      cname.set_title(title, fontsize=8)
                      cb_cross = fig.colorbar(var_fill, ax=cname)
                      cb_cross.ax.tick_params(labelsize=6)
                      s1 = cname.streamplot(to_np(x), to_np(y),
                                            hori,
                                            ver,
                                            color = 'k')
                      cname.set_ylim([2000, 7000])
                    
        ####### CALL cross plot function for every var of CONT###############
        print(np.max(nice))
        crossplot(ax_nice,xs,ys,nice,stream,dopp,np.arange(0., 1, .05),"autumn_r",'CONT change in ice number from collection %02d' % (i) + '%02d' % (timeout) + 'UTC, g/kg')
        crossplot(ax_qc_d,xs,ys,qc_d,stream,dopp,np.arange(0,.1,.01),"BuGn",'CONT contact freezing droplet %02d' % (i) + '%02d' % (timeout) + 'UTC, kg -1')
        crossplot(ax_qi_d,xs,ys,qi_d,stream,dopp,np.arange(0,1e-3, 1e-4),"BuGn",'CONT immersion freezing droplet %02d' % (i) + '%02d' % (timeout) + 'UTC, kg -1')
        crossplot(ax_qc_r,xs,ys,qc_r,stream,dopp,np.arange(0,1e-3,.1e-4),"RdPu",'CONT contact freezing rain %02d' % (i) + '%02d' % (timeout) + 'UTC, g/hr')
        crossplot(ax_qi_r,xs,ys,qi_r,stream,dopp,np.arange(0,1e-3,.1e-4),"YlGnBu",'CONT immersion freezing rain %02d' % (i) + '%02d' % (timeout) + 'UTC, g/kg')

        #
        crossplot(ax_nice_t,xs,ys,nice_t,stream,dopp_t,np.arange(0., 1, .05),"autumn_r",'TURB change in ice number from collection %02d' % (i) + '%02d' % (timeout) + 'UTC')   
        crossplot(ax_qc_d_t,xs,ys,qc_d_t,stream,dopp_t,np.arange(0,1e-3,1e-4),"BuGn",'TURB contact freezing droplet %02d' % (i) + '%02d' % (timeout) + 'UTC')
        crossplot(ax_qi_d_t,xs,ys,qi_d_t,stream,dopp_t,np.arange(0,1e-3,1e-4),"BuGn",'TURB immersion freezing droplet %02d' % (i) + '%02d' % (timeout) + 'UTC')
        crossplot(ax_qc_r_t,xs,ys,qc_r_t,stream,dopp_t,np.arange(0,1e-3,1e-4),"RdPu",'TURB contact freezing rain %02d' % (i) + '%02d' % (timeout) + 'UTC')
        crossplot(ax_qi_r_t,xs,ys,qi_r_t,stream,dopp_t,np.arange(0,1e-3,1e-4),"YlGnBu",'TURB immersion freezing rain %02d' % (i) + '%02d' % (timeout) + 'UTC')
        
        #
        acclevel = np.arange(-1, 1, .05)
        crossplot(ax_nice_diff,xs,ys,nice_diff,stream,dopp,np.arange(-0.1, .11, .01),"seismic",'change in ice number from collection diffrence %02d' % (i) + '%02d' % (timeout) + 'UTC')
        crossplot(ax_qc_d_diff,xs,ys,qc_d_diff,stream,dopp,np.arange(-.02, .022,.002),"seismic",'contact freezing droplet diffrence %02d' % (i) + '%02d' % (timeout) + 'UTC')
        crossplot(ax_qi_d_diff,xs,ys,qi_d_diff,stream,dopp,np.arange(-1e-3,1e-3,1e-4),"seismic",'immersion freezing droplet difference %02d' % (i) + '%02d' % (timeout) + 'UTC')
        crossplot(ax_qc_r_diff,xs,ys,qc_r_diff,stream,dopp,np.arange(-1e-3,1e-3,1e-4),"seismic",'contact freezing rain deposition diffrence %02d' % (i) + '%02d' % (timeout) + 'UTC')
        crossplot(ax_qi_r_diff,xs,ys,qi_r_diff,stream,dopp,np.arange(-1e-3,1e-3,1e-4),"seismic",'immersion freezing rain difference %02d' % (i) + '%02d' % (timeout) + 'UTC')
        
        
        print('saving%02d' % (i) + '%02d' % (timeout) + 'UTC...')
        #pyplot.savefig('cross_sec/ABS_YSUmed_comparediff_ref_ice%02d'% (i) + '%02d' % (timeout) + '.png', facecolor = 'w')
        #pyplot.savefig('1900_fixref.png', facecolor = 'w')

NameError: name 'cross_start' is not defined